### Эксперименты с моделькой

- Как и с EDA, потыкаться всем в модельки: логистическая регрессия, KNN, Naive Bayes (это базово + три строки кода на каждую в самом простом варианте) и посчитать метрики

- Для векторизации пока взять два варианта: 1. мешок слов 2. все то же самое с заменой мешка слов на tf-idf